In [1]:
import os
os.chdir("../tsp")
print("Current Working Directory:", os.getcwd())
import time
import torch
from torch.distributions import Categorical, kl
from d2l.torch import Animator

from net import Net
from aco import ACO
from utils import gen_pyg_data, load_test_dataset

torch.manual_seed(12345)

EPS = 1e-10
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

Current Working Directory: /Users/erostrate9/Desktop/CSI5137B test/project/code/DeepACO/tsp


In [2]:
@torch.no_grad()
def infer_instance(model, pyg_data, distances, n_ants, t_aco_diff, k_sparse=None):
    if model:
        model.eval()
        heu_vec = model(pyg_data)
        heu_mat = model.reshape(pyg_data, heu_vec) + EPS
    
        aco = ACO(
        n_ants=n_ants,
        heuristic=heu_mat,
        distances=distances,
        device=device
        )
    
    else:
        aco = ACO(
        n_ants=n_ants,
        distances=distances,
        device=device
        )
        if k_sparse:
            aco.sparsify(k_sparse)
        
    results = torch.zeros(size=(len(t_aco_diff),), device=device)
    for i, t in enumerate(t_aco_diff):
        best_cost = aco.run(t)
        results[i] = best_cost
    return results
        
    
@torch.no_grad()
def test(dataset, model, n_ants, t_aco, k_sparse=None):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco_diff),), device=device)
    start = time.time()
    for pyg_data, distances in dataset:
        results = infer_instance(model, pyg_data, distances, n_ants, t_aco_diff, k_sparse)
        sum_results += results
    end = time.time()
    
    return sum_results / len(dataset), end-start

In [4]:
import os

directory_path = "../data/tsp/TSPLIB"
tsp_files = [f for f in os.listdir(directory_path) if f.endswith(".tsp")]
print(tsp_files)

['pr439.tsp', 'rd100.tsp', 'rl5934.tsp', 'pcb442.tsp', 'u2319.tsp', 'gil262.tsp', 'eil51 copy.tsp', 'pcb3038.tsp', 'lin105.tsp', 'fl417.tsp', 'tsp225.tsp', 'fl1400.tsp', 'nrw1379.tsp', 'd2103.tsp', 'kroA150.tsp', 'pcb1173.tsp', 'd198.tsp', 'fl1577.tsp', 'ch130.tsp', 'kroB100.tsp', 'u1060.tsp', 'berlin52.tsp', 'eil51.tsp', 'rl1304.tsp', 'u2152.tsp', 'u724.tsp', 'kroD100.tsp', 'pr299.tsp', 'berlin52 copy.tsp', 'rd400.tsp', 'vm1084.tsp', 'rat575.tsp', 'd1655.tsp', 'ch150.tsp', 'd15112.tsp', 'pr107.tsp', 'kroB200.tsp', 'brd14051.tsp', 'a280.tsp', 'd1291.tsp', 'pr264.tsp', 'pr76.tsp', 'kroA100 copy.tsp', 'd493.tsp', 'pr136.tsp', 'rat195.tsp', 'kroA200 copy.tsp', 'rl11849.tsp', 'kroA100.tsp', 'kroB150.tsp', 'bier127.tsp', 'kroC100.tsp', 'usa13509.tsp', 'pr439 copy.tsp', 'eil76.tsp', 'pr124.tsp', 'rl1323.tsp', 'p654.tsp', 'rl1889.tsp', 'd657.tsp', 'eil101.tsp', 'fnl4461.tsp', 'pr2392.tsp', 'rat783.tsp', 'ts225.tsp', 'u1432.tsp', 'u1817.tsp', 'lin318.tsp', 'd18512.tsp', 'rl5915.tsp', 'st70.tsp

### Test on TSP20

DeepACO

In [3]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, k_sparse, device)
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/tsp/tsp{n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco, k_sparse)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

total duration:  153.85154390335083
T=1, average cost is 3.9320340156555176.
T=10, average cost is 3.8233907222747803.
T=20, average cost is 3.816669225692749.
T=30, average cost is 3.8122851848602295.
T=40, average cost is 3.8114993572235107.
T=50, average cost is 3.8111512660980225.
T=100, average cost is 3.809321165084839.


ACO

In [4]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, k_sparse, device)
avg_aco_best, duration = test(test_list, None, n_ants, t_aco, k_sparse)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

total duration:  146.5802402496338
T=1, average cost is 5.254907608032227.
T=10, average cost is 4.2066216468811035.
T=20, average cost is 4.0416178703308105.
T=30, average cost is 3.9487416744232178.
T=40, average cost is 3.911342144012451.
T=50, average cost is 3.8893744945526123.
T=100, average cost is 3.8348400592803955.
